# Data

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"amaydoshi","key":"a294ac6bbd89b1a35a467a969ad24899"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download learning-agency-lab-automated-essay-scoring-2
! unzip learning-agency-lab-automated-essay-scoring-2.zip

 42% 5.00M/11.9M [00:00<00:00, 39.6MB/s]
100% 11.9M/11.9M [00:00<00:00, 82.7MB/s]
Archive:  learning-agency-lab-automated-essay-scoring-2.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# EDA

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.head()

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3


# Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Sample DataFrame (replace this with your actual DataFrame)
# df = pd.DataFrame({'full_text': [...], 'score': [...]})
# Example:
# df = pd.DataFrame({
#     'full_text': ["Sample text 1", "Sample text 2", "More sample text", "Another text"],
#     'score': [1, 2, 3, 6]
# })

# Preprocess the target labels
label_encoder = LabelEncoder()
df['score'] = label_encoder.fit_transform(df['score'])  # Encodes scores 1-6 to 0-5
num_classes = df['score'].nunique()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(df['full_text'], df['score'], test_size=0.2, random_state=42)

# Tokenize and pad text data
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

max_length = 100  # Maximum length of sequences
X_train_sequences = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_length, padding="post")
X_test_sequences = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_length, padding="post")

# Build the model
model = tf.keras.Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=max_length),
    LSTM(64, return_sequences=True),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # num_classes = 6 for scores 1-6
])

# Compile the model
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Train the model
epochs = 50  # Adjust based on dataset size and performance
history = model.fit(X_train_sequences, y_train,
                    epochs=epochs,
                    validation_data=(X_test_sequences, y_test),
                    batch_size=32)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_sequences, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


433/433 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.3344 - loss: 1.5448 - val_accuracy: 0.4012 - val_loss: 1.3354
Epoch 2/50
433/433 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.4134 - loss: 1.3031 - val_accuracy: 0.4128 - val_loss: 1.2657
Epoch 3/50
433/433 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.4890 - loss: 1.1713 - val_accuracy: 0.4200 - val_loss: 1.2732
Epoch 4/50
433/433 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.5383 - loss: 1.0732 - val_accuracy: 0.4136 - val_loss: 1.3112
Epoch 5/50
433/433 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5857 - loss: 0.9846 - val_accuracy: 0.4105 - val_loss: 1.3547
Epoch 6/50
433/433 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.6438 - loss: 0.8733 - val_accuracy: 0.4058 - val_loss: 1.4833
Epoch 7/50
433/433 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6921 - loss: 0.7789 - val_accuracy: 0.4035 - val_loss: 1.6439
Epoch 8/50
433/433 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7485 - loss: 0.6559 - val_accuracy: 0.3897 - v

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [ ]:
# Sample DataFrame for example (replace with actual essay data)
# Assume df['essay'] contains the essay text and df['score'] has scores (1-6).
# df = pd.DataFrame({
#     'essay': ["Essay text example 1", "Another example of essay text", "Sample essay text", "More essay content"],
#     'score': [3, 5, 2, 6]
# })

# Encode the target labels
label_encoder = LabelEncoder()
df['score'] = label_encoder.fit_transform(df['score'])  # Encodes 1-6 to 0-5
num_classes = df['score'].nunique()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(df['full_text'], df['score'], test_size=0.2, random_state=42)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and encode sequences for BERT
max_length = 512  # Adjust based on essay length if needed

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def tokenize_texts(texts):
    return tokenizer(
        texts.tolist(),
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors="tf"
    )

In [ ]:
X_train_encodings = tokenize_texts(X_train)
X_test_encodings = tokenize_texts(X_test)

# Convert labels to tensors
y_train_tensor = tf.convert_to_tensor(y_train)
y_test_tensor = tf.convert_to_tensor(y_test)

In [ ]:
# Load BERT model for sequence classification
model2 = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)

# Compile the model
model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=["accuracy"])

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Train the model
epochs = 3  # BERT typically fine-tunes well within 2-4 epochs
history = model2.fit(
    {'input_ids': X_train_encodings['input_ids'], 'attention_mask': X_train_encodings['attention_mask']},
    y_train_tensor,
    validation_data=(
        {'input_ids': X_test_encodings['input_ids'], 'attention_mask': X_test_encodings['attention_mask']},
        y_test_tensor
    ),
    epochs=epochs,
    batch_size=8  # Adjust based on available memory
)

Epoch 1/3


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/tf_keras/src/engine/training.py", line 1381, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/tf_keras/src/engine/training.py", line 1370, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_tf_utils.py", line 1706, in train_step
        loss = self.compiled_loss(y, y_pred, sample_weight, regularization_losses=self.losses)
    File "/usr/local/lib/python3.10/dist-packages/tf_keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/usr/local/lib/python3.10/dist-packages/tf_keras/src/losses.py", line 143, in __call__


In [ ]:
# Convert labels to appropriate shape and data type
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.int32)
y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.int32)

# Compile the model with SparseCategoricalCrossentropy
model2.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

# Train the model
epochs = 1  # BERT typically fine-tunes well within 2-4 epochs
history = model2.fit(
    {'input_ids': X_train_encodings['input_ids'], 'attention_mask': X_train_encodings['attention_mask']},
    y_train_tensor,
    validation_data=(
        {'input_ids': X_test_encodings['input_ids'], 'attention_mask': X_test_encodings['attention_mask']},
        y_test_tensor
    ),
    epochs=epochs,
    batch_size=8  # Adjust based on available memory
)


1731/1731 [==============================] - 1853s 1s/step - loss: 1.5879 - accuracy: 0.3159 - val_loss: 1.5638 - val_accuracy: 0.3654


# Prediction

In [ ]:
# Grading Pipeline Function
def grade_essay(essay_text):
    """
    Preprocesses the essay text, makes a prediction, and outputs a grade.
    """
    # Tokenize and encode the input essay
    encodings = tokenizer(
        [essay_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors="tf"
    )

    # Predict the score
    predicted_score = model2(encodings)
    predicted_label = tf.argmax(predicted_score.logits, axis=-1).numpy()

    # Convert back to original score label
    grade = label_encoder.inverse_transform(predicted_label)[0]
    return grade

In [ ]:
# Test the grading pipeline
sample_essay = "This is a sample essay text that we want to grade based on quality and content."
predicted_grade = grade_essay(sample_essay)
print(f"Predicted Grade: {predicted_grade}")

Predicted Grade: 2
